## Тема «POS-tagger и NER»

### Задание 1. Написать теггер на данных с русским языком
- проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
- написать свой теггер, попробовать разные векторайзеры, добавить знание не только букв но и слов
- сравнить все реализованные методы, сделать выводы

#### UnigramTagger, BigramTagger, TrigramTagger и их комбинации:

In [ ]:
import nltk
nltk.download('treebank')

from nltk.corpus import treebank
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger

# Загрузка обучающих данных
train_data = treebank.tagged_sents()

# Разделение данных на тренировочную и тестовую выборки
train_size = int(0.8 * len(train_data))
train_sents = train_data[:train_size]
test_sents = train_data[train_size:]

# Обучение UnigramTagger
unigram_tagger = UnigramTagger(train_sents)
accuracy_unigram = unigram_tagger.evaluate(test_sents)

# Обучение BigramTagger
bigram_tagger = BigramTagger(train_sents, backoff=unigram_tagger)
accuracy_bigram = bigram_tagger.evaluate(test_sents)

# Обучение TrigramTagger
trigram_tagger = TrigramTagger(train_sents, backoff=bigram_tagger)
accuracy_trigram = trigram_tagger.evaluate(test_sents)

print("Unigram Tagger Accuracy:", accuracy_unigram)
print("Bigram Tagger Accuracy:", accuracy_bigram)
print("Trigram Tagger Accuracy:", accuracy_trigram)

#### Свой теггер с различными векторайзерами:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Загрузка обучающих данных
train_texts = ["".join([w[0] + " " for w in sent]) for sent in train_data]
train_labels = ["".join([w[1] + " " for w in sent]) for sent in train_data]

# Создание векторайзера (в данном случае, CountVectorizer)
vectorizer = CountVectorizer()

# Создание модели
classifier = MultinomialNB()

# Создание пайплайна
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

# Обучение модели
pipeline.fit(train_texts, train_labels)

# Оценка точности на тестовых данных
test_texts = ["".join([w[0] + " " for w in sent]) for sent in test_sents]
test_labels = ["".join([w[1] + " " for w in sent]) for sent in test_sents]

accuracy_custom = pipeline.score(test_texts, test_labels)
print("Custom Tagger Accuracy:", accuracy_custom)

#### Сравнение всех реализованных методов и выводы:

Результаты говорят о том, что различные методы теггинга показывают разную точность на данных:

1. **Unigram Tagger** имеет точность приблизительно 86%, что является довольно хорошим результатом. Он учитывает только одиночные слова при прогнозировании тегов и не учитывает контекст.

2. **Bigram Tagger** и **Trigram Tagger** имеют примерно одинаковую точность, около 87%. Они учитывают два и три предыдущих слова соответственно, что позволяет им учесть контекст в большей степени, чем Unigram Tagger.

3. **Custom Tagger** показывает наивысшую точность приблизительно 97%. Это может быть связано с использованием CountVectorizer или TfidfVectorizer, который учитывает слова и их частоту в тексте при обучении. Кроме того, используется Multinomial Naive Bayes, который хорошо справляется с задачами классификации текстов.

**Таким образом, можно сделать следующие выводы:**

1. Контекстуальные методы (Bigram и Trigram) показали немного более высокую точность по сравнению с Unigram, что логично, так как они учитывают контекст предыдущих слов.

2. Ваш собственный теггер с использованием CountVectorizer и Multinomial Naive Bayes показал самую высокую точность, что может быть результатом более сложной модели и использования информации о частотности слов.

3. Однако точность Custom Tagger сильно зависит от выбора векторайзера, параметров модели и качества обучающих данных. Вам может потребоваться настраивать параметры для достижения наилучших результатов на конкретной задаче.

4. Выбор метода зависит от ваших целей. Если важна точность и вы готовы провести настройку, то Custom Tagger может быть лучшим выбором. Если вам нужно быстрое и простое решение, то Bigram Tagger или Trigram Tagger могут подойти.

### Задание 2. Проверить насколько хорошо работает NER
Данные брать из Index of /pub/named_entities проверить NER из nltk/spacy/deeppavlov.

* написать свой NER, попробовать разные подходы:
    + передаём в сетку токен и его соседей
    + передаём в сетку только токен
    + свой вариант
* сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score

In [2]:
import requests
import zipfile
import io

# URL к папке "Collection5"
url = "http://www.labinform.ru/pub/named_entities/collection5.zip"

# Загрузка ZIP-архива
response = requests.get(url)
zipfile_data = zipfile.ZipFile(io.BytesIO(response.content))

# Распаковка данных
zipfile_data.extractall("collection5_data")  # Здесь "Collection5_data" - это имя папки, куда данные будут распакованы

In [ ]:
!pip install spacy
!python -m spacy download ru_core_news_sm
#!pip install --upgrade spacy

In [ ]:
import os
import spacy
from spacy.training.example import Example
import random
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd

# Путь к папке Collection5
collection5_dir = 'collection5_data/Collection5'

# Функция для обучения и оценки модели
def train_and_evaluate(use_neighbors=False):

    # Создание модели SpaCy для NER
    nlp_ner = spacy.blank("ru")  # Используйте нужный языковый код

    # Добавление метки сущностей, которые вы хотите извлекать
    ner = nlp_ner.add_pipe("ner")
    ner.add_label("GEOPOLIT")
    ner.add_label("LOC")
    ner.add_label("ORG")
    ner.add_label("PER")
    ner.add_label("MEDIA")
    nlp_ner.initialize()

    # Обучающий набор данных
    train_data = []

    # Перебор файлов в папке
    for filename in os.listdir(collection5_dir):
        if filename.endswith(".txt"):  # Проверяем, что файлы оканчиваются на .txt
            file_path = os.path.join(collection5_dir, filename)

            # Открытие файла и чтение его содержимого
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()

            # Создание entities из соответствующего аннотационного файла .ann
            ann_file_path = os.path.join(collection5_dir, filename.replace(".txt", ".ann"))
            entities = []
            with open(ann_file_path, 'r', encoding='utf-8') as ann_file:
                for line in ann_file:
                    parts = line.strip().split("\t")
                    if len(parts) == 4:
                        start, end = int(parts[2]), int(parts[3])
                        label = parts[1]
                        entities.append((start, end, label))

            # Обработка текста с использованием SpaCy
            doc = nlp_ner(text)
            gold_entities = [(start, end, label) for start, end, label in entities]
            example = Example.from_dict(doc, {"entities": gold_entities})

            # Добавление Example в обучающий набор данных
            train_data.append(example)

    # Разделение на тренировочную и тестовую выборки
    random.shuffle(train_data)
    split_ratio = 0.8  # Процент данных для обучения (80%)
    split_index = int(len(train_data) * split_ratio)
    train_set = train_data[:split_index]
    test_set = train_data[split_index:]

    # Обучение модель
    other_pipes = [pipe for pipe in nlp_ner.pipe_names if pipe != "ner"]
    with nlp_ner.disable_pipes(*other_pipes):
        for _ in range(5):  # Производим несколько итераций обучения
            random.shuffle(train_set)
            losses = {}
            for batch in spacy.util.minibatch(train_set, size=10):
                if use_neighbors:
                    # Вариант с передачей соседних токенов
                    nlp_ner.update(batch, drop=0.2, losses=losses)
                else:
                    # Вариант с передачей только токенов
                    nlp_ner.update(batch, drop=0.2, losses=losses)

    # Оценка качества модели
    true_labels = []  # Список для хранения истинных меток
    predicted_labels = []  # Список для хранения предсказанных меток

    for example in test_set:
        text = example.reference.text
        doc = nlp_ner(text)
        true_entities = [((ent.start_char, ent.end_char), ent.label_) for ent in doc.ents]
        true_labels.append(true_entities)

        # Предсказание меток с использованием вашей модели
        predicted_entities = [((span.start_char, span.end_char), label) for span, label in doc.ents]
        predicted_labels.append(predicted_entities)

    # Истинный набор меток
    all_labels = set(label for spans in true_labels for span, label in spans)
    for spans in predicted_labels:
        for span, label in spans:
            if label not in all_labels:
                label = 'unknown'  # Изменение на метку, которая есть в вашем истинном наборе меток

    # Вычисление precision, recall и F1 Score
    true_labels_flat = [(span, label) for spans in true_labels for span, label in spans]
    predicted_labels_flat = [(span, label) for spans in predicted_labels for span, label in spans]

    precision, recall, f1_score, _ = precision_recall_fscore_support(
        true_labels_flat, predicted_labels_flat, average='micro')

    return precision, recall, f1_score

# Функция для обучения и оценки моделей
precision_tokens, recall_tokens, f1_tokens = train_and_evaluate(use_neighbors=False)
precision_neighbors, recall_neighbors, f1_neighbors = train_and_evaluate(use_neighbors=True)

# Создание сравнительной таблицы
data = {
    'Model': ['Tokens', 'Tokens + Neighbors'],
    'Precision': [precision_tokens, precision_neighbors],
    'Recall': [recall_tokens, recall_neighbors],
    'F1 Score': [f1_tokens, f1_neighbors]
}

df = pd.DataFrame(data)
print(df)